## Extract Data

In [ ]:
!gdown 1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV
!unzip -q icdar2003.zip -d datasets

Downloading...
From (original): https://drive.google.com/uc?id=1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV
From (redirected): https://drive.google.com/uc?id=1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV&confirm=t&uuid=7f6cf8d6-42ed-4b14-9191-fbae8ef1e29f
To: /content/icdar2003.zip
100% 45.7M/45.7M [00:01<00:00, 32.4MB/s]


In [ ]:
import xml.etree.ElementTree as ET

path = './datasets/SceneTrialTrain/words.xml'
tree = ET.parse(path)
root = tree.getroot()
root

<Element 'tagset' at 0x78635c8a2f20>

In [ ]:
print(root[:10])

[<Element 'image' at 0x78635c8a2f70>, <Element 'image' at 0x78635c8a3ba0>, <Element 'image' at 0x78635c8a3e20>, <Element 'image' at 0x78635c8c0950>, <Element 'image' at 0x78635c8c1bc0>, <Element 'image' at 0x78635c8c24d0>, <Element 'image' at 0x78635c8c2de0>, <Element 'image' at 0x78635c8c3ab0>, <Element 'image' at 0x78635c8e0a90>, <Element 'image' at 0x78635c8e12b0>]


In [ ]:
i = 0
for im in root:
    print(im[0].text)
    print(im[1].attrib)
    i += 1
    if i == 5:
        break

apanar_06.08.2002/IMG_1261.JPG
{'x': '1600', 'y': '1200'}
apanar_06.08.2002/IMG_1263.JPG
{'x': '1600', 'y': '1200'}
apanar_06.08.2002/IMG_1265.JPG
{'x': '1600', 'y': '1200'}
apanar_06.08.2002/IMG_1269.JPG
{'x': '1600', 'y': '1200'}
apanar_06.08.2002/IMG_1281.JPG
{'x': '1600', 'y': '1200'}


In [25]:
for im in root:
    im_name = im[0].text
    for bboxes in im.findall('taggedRectangles'):
        for bbox in bboxes:
            print(bbox[0].text)
            print(bbox.attrib)
    break

self
{'x': '174.0', 'y': '392.0', 'width': '274.0', 'height': '195.0', 'offset': '0.0', 'rotation': '0.0', 'userName': 'admin'}
adhesive
{'x': '512.0', 'y': '391.0', 'width': '679.0', 'height': '183.0', 'offset': '0.0', 'rotation': '0.0', 'userName': 'admin'}
address
{'x': '184.0', 'y': '612.0', 'width': '622.0', 'height': '174.0', 'offset': '-2.0', 'rotation': '0.0', 'userName': 'admin'}
labels
{'x': '863.0', 'y': '599.0', 'width': '446.0', 'height': '187.0', 'offset': '0.0', 'rotation': '0.0', 'userName': 'admin'}
36
{'x': '72.0', 'y': '6.0', 'width': '95.0', 'height': '87.0', 'offset': '0.0', 'rotation': '0.0', 'userName': 'admin'}
89m
{'x': '247.0', 'y': '2.0', 'width': '197.0', 'height': '88.0', 'offset': '0.0', 'rotation': '0.0', 'userName': 'admin'}
cls
{'x': '792.0', 'y': '0.0', 'width': '115.0', 'height': '81.0', 'offset': '0.0', 'rotation': '0.0', 'userName': 'admin'}
250
{'x': '200.0', 'y': '848.0', 'width': '228.0', 'height': '139.0', 'offset': '0.0', 'rotation': '0.0', 'us

In [26]:
def extract_data(path):
    tree = ET.parse(path)
    root = tree.getroot()

    im_paths, im_sizes, im_labels, im_bboxes = [], [], [], []
    for im in root:
        bboxes = []
        labels = []

        for bbs in im.findall('taggedRectangles'):
            for bb in bbs:
                if not bb[0].text.isalnum():
                    continue
                if "é" in bb[0].text.lower() or "ñ" in bb[0].text.lower():
                    continue

                bboxes.append(
                    [float(bb.attrib['x']),
                     float(bb.attrib['y']),
                     float(bb.attrib['width']),
                     float(bb.attrib['height'])]
                )
                labels.append(bb[0].text.lower())

        im_paths.append(im[0].text)
        im_sizes.append((int(im[1].attrib['x']), int(im[1].attrib['y'])))
        im_labels.append(labels)
        im_bboxes.append(bboxes)

    return im_paths, im_sizes, im_labels, im_bboxes

In [28]:
im_paths, im_sizes, im_labels, im_bboxes = extract_data(path)
print(im_paths[0])
print(im_sizes[0])
print(im_labels[0])
print(im_bboxes[0])

apanar_06.08.2002/IMG_1261.JPG
(1600, 1200)
['self', 'adhesive', 'address', 'labels', '36', '89m', 'cls', '250', 'on', 'a', 'roll']
[[174.0, 392.0, 274.0, 195.0], [512.0, 391.0, 679.0, 183.0], [184.0, 612.0, 622.0, 174.0], [863.0, 599.0, 446.0, 187.0], [72.0, 6.0, 95.0, 87.0], [247.0, 2.0, 197.0, 88.0], [792.0, 0.0, 115.0, 81.0], [200.0, 848.0, 228.0, 139.0], [473.0, 878.0, 165.0, 109.0], [684.0, 878.0, 71.0, 106.0], [806.0, 844.0, 218.0, 141.0]]


## Convert to YOLO format

In [29]:
def convert_to_yolo_format(im_paths, im_sizes, im_bboxes):
    yolo_data = []
    for im_path, im_size, im_bbox in zip(im_paths, im_sizes, im_bboxes):
        im_width, im_height = im_size
        yolo_labels = []

        for bbox in im_bbox:
            x, y, w, h = bbox # (topleft_x, topleft_y, width, height)
            x_center = (x + w / 2) / im_width
            y_center = (y + h / 2) / im_height
            w /= im_width
            h /= im_height
            class_id = 0

            yolo_labels.append(f'{class_id} {x_center} {y_center} {w} {h}')
        yolo_data.append((im_path, yolo_labels))
    return yolo_data

In [32]:
class_labels = ['text']
yolo_data = convert_to_yolo_format(im_paths, im_sizes, im_bboxes)
yolo_data[:3]

[('apanar_06.08.2002/IMG_1261.JPG',
  ['0 0.194375 0.40791666666666665 0.17125 0.1625',
   '0 0.5321875 0.40208333333333335 0.424375 0.1525',
   '0 0.309375 0.5825 0.38875 0.145',
   '0 0.67875 0.5770833333333333 0.27875 0.15583333333333332',
   '0 0.0746875 0.04125 0.059375 0.0725',
   '0 0.2159375 0.03833333333333333 0.123125 0.07333333333333333',
   '0 0.5309375 0.03375 0.071875 0.0675',
   '0 0.19625 0.7645833333333333 0.1425 0.11583333333333333',
   '0 0.3471875 0.7770833333333333 0.103125 0.09083333333333334',
   '0 0.4496875 0.7758333333333334 0.044375 0.08833333333333333',
   '0 0.571875 0.7620833333333333 0.13625 0.1175']),
 ('apanar_06.08.2002/IMG_1263.JPG',
  ['0 0.5346875 0.6008333333333333 0.529375 0.14333333333333334']),
 ('apanar_06.08.2002/IMG_1265.JPG',
  ['0 0.566875 0.21875 0.31375 0.0675',
   '0 0.56 0.22375 0.3375 0.025833333333333333',
   '0 0.5709375 0.28458333333333335 0.255625 0.0575',
   '0 0.5771875 0.33708333333333335 0.165625 0.060833333333333336',
   '0 0.